In [7]:
import re
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
df_all = pd.read_pickle('./data/df_all_lemma.pkl')
print(len(df_all))
df_all.head(1)

2524


,comments,description,duration,event,film_date,languages,link,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,transcript,url,views
0,4553.0,Sir Ken Robinson makes an entertaining and pro...,1164.0,TED2006,1.140826e+09,60.0,NaN,Ken Robinson,Ken Robinson: Do schools kill creativity?,1.0,1.151367e+09,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,it been great hasnt it. ive been blown away by...,https://www.ted.com/talks/ken_robinson_says_sc...,47227110.0


In [9]:
docs_all = list(df_all['transcript'])

from nltk.corpus import stopwords

stops_standard = stopwords.words('english')
stops_custom = ['shes','youll','ill','yeah','th','yes','oh',
                'ok','okay','might','ha','mr','bg','ms',
                'mrs','ca','em','da','ted','pm','hey','al']+[re.sub('[^A-Za-z ]+', '', w) for w in stops_standard]
stop_list = list(set(stops_standard + stops_custom))

print(stop_list)

['wouldn', 'your', 'ca', 've', 'pm', 'should', 'why', 'ted', 'after', 'neednt', 'themselves', 'it', 'now', 'while', 'are', 'ok', 'wont', 'don', 'to', 'didnt', 'hasnt', 'yourself', 'yourselves', 'once', 'below', 'or', "you'll", 'shant', 'you', 'as', 'between', "isn't", "mustn't", 'how', 'out', 'have', "won't", 'of', "shan't", 'mrs', 'mustnt', 'them', 'from', 'him', 'those', 'further', 'such', 'too', 'ain', 'al', 'in', 'th', 'off', 'with', 'my', 'youre', "mightn't", 'more', 'em', 'our', 'haven', 'against', 'most', 'for', 'youve', 'was', 'wouldnt', 'we', 'only', 'oh', 'mightnt', 'who', 'just', 'here', 'under', 'doesnt', 'very', "hadn't", 'm', "aren't", 'myself', 'during', 'what', 'is', 'when', "wouldn't", 'over', 'isn', 'wasn', 'down', 'weren', 'might', "wasn't", 'then', 'shouldve', 'shes', 'dont', 'above', "that'll", 'up', 'me', 'about', 'his', 'no', 'll', 'own', 're', 'having', 'yes', 's', 'herself', "doesn't", 'any', 'whom', 'because', 'won', 'y', 'were', 'few', 'okay', 'i', 'ms', 'the

In [30]:
docs = docs_all

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import Normalizer


n_samples = 2000
n_features = 2000
n_components = 100
n_top_words = 10
n_gram = 1
alpha = 0.1
stop_choice= stop_list

max_df = 0.3
min_df = 5
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for LSA.
print("Extracting tf-idf features for LSA...")
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,
                                   ngram_range=(n_gram,n_gram),
                                   max_features=n_features,
                                   stop_words=stop_choice)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(docs)
print("done in %0.3fs." % (time() - t0))

# Fit the LDA model
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lsa = TruncatedSVD(n_components=n_components, algorithm = 'randomized',random_state=1)
t0 = time()
lsa.fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LSA model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lsa, tfidf_feature_names, n_top_words)

Extracting tf-idf features for LSA...
done in 2.689s.
Fitting LDA models with tf features, n_samples=2000 and n_features=2000...
done in 0.437s.

Topics in LSA model:
Topic #0: city brain data water community design book company space computer
Topic #1: brain cell animal planet water cancer earth robot universe light
Topic #2: city water planet energy ocean earth climate space oil sea
Topic #3: cancer cell patient disease health drug government africa dollar data
Topic #4: data computer robot information machine company design internet government phone
Topic #5: city design cancer patient cell car architecture street neighborhood space
Topic #6: brain city neuron cortex memory region area neighborhood street activity
Topic #7: robot water animal food fish machine plant leg girl ocean
Topic #8: music sound song food play water africa plant business musician
Topic #9: music robot cell war cancer democracy government sound song political
Topic #10: music patient car universe sound health 

In [31]:
X = lsa.fit_transform(tfidf)
X = Normalizer(copy=False).fit_transform(X)


In [32]:
# np.savetxt('./data/lsa_data_100.tsv', X, delimiter='\t',newline='\n')
# Xlabel = list(df_all.title)
# with open('./data/lsa_meta_100.tsv', 'w') as file:
#     for label in Xlabel:
#         file.write(label+'\n')

### Find related talks

In [21]:
import sklearn.neighbors.kd_tree as kdtree

In [22]:
tree = kdtree.KDTree(X, leaf_size=10) 
N = 1
dist, ind = tree.query([X[N]], k=20)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1 	 Averting the climate crisis
2370 	 A climate solution where all sides can win
1977 	 Climate change is happening. Here's how we adapt
1970 	 Why climate change is a threat to human rights
210 	 New thinking on the climate crisis
157 	 A critical look at geoengineering against climate change
108 	 Salvation (and profit) in greentech
2077 	 This country isn't just carbon neutral -- it's carbon negative
2063 	 The case for optimism on climate change
2105 	 The inside story of the Paris climate agreement
2114 	 A provocative way to finance the fight against climate change
1267 	 Let's prepare for our new climate
1103 	 Why I must speak out about climate change
958 	 The shareable future of cities
2178 	 A small country with big ideas to get rid of fossil fuels
1744 	 The state of the climate — and what we might do about it
723 	 Let the environment guide our development
552 	 Global ethic vs. national interest
436 	 What comes after An Inconvenient Truth?
442 	 The business logic of su

In [23]:
df_related = pd.DataFrame(eval(df_all.iloc[N]['related_talks']))
for k in range(len(df_related)):
    print(list(df_all[df_all['title']==df_related.iloc[k]['title']].title))
#     print(df_all[df_all['title']==df_related.iloc[k]['title']].views)
#     print(list(df_all[df_all['title']==df_related.iloc[k]['title']].tags))

['New thinking on the climate crisis']
['The business logic of sustainability']
['The state of the climate — and what we might do about it']
['A climate solution where all sides can win']
["Climate change is happening. Here's how we adapt"]
['Why climate change is a threat to human rights']
